In [5]:
import re
import json
import numpy as np
import pandas as pd

In [6]:
def get_type(rel_extend_map, rel):
    for key, val in rel_extend_map.items():
        for v in val:
            if v == rel:
                return key


def get_count_matrix(rel_extend_map, merged_golds, backward):
    value = [c for v in rel_extend_map.values() for c in v]
    dict = {k: [] for k in value}
    dict_count = {k: [] for k in value}

    for item in backward:
        rel = item[0][1]
        dict[rel].append(item)

    for k,v in dict.items():   
        cur_extend_rel = k
        cur_list = v
        for item in cur_list:
            count_correct = 0
            count_wrong_from_gold = 0
            count_wrong_from_pred = 0

            cur_gold_rel = get_type(rel_extend_map, cur_extend_rel)
            cur_idx = item[0][0]

            cur_merged_golds = merged_golds[cur_idx]
            if type(cur_merged_golds) == tuple:
                cur_golds = [cur_merged_golds]
            else:
                cur_golds = [gold for gold in cur_merged_golds if gold[1] == cur_gold_rel]
            cur_preds = [pred for pred in item] 

            preds_length = len(cur_preds)
            golds_length = len(cur_golds)

            if golds_length != 0:
                for pred in cur_preds:
                    for cmp_gold in cur_golds:
                        if pred[2] == '' and pred[3] == '':
                            preds_length -= 1
                            break
                        elif pred[2] == '' or pred[3] == '':
                            continue
                        elif (pred[2] in cmp_gold[2] and pred[3] in cmp_gold[3]) \
                            or (cmp_gold[2] in pred[2] and cmp_gold[3] in pred[3]) \
                            or (pred[2] in cmp_gold[2] and cmp_gold[3] in pred[3]) \
                            or (cmp_gold[2] in pred[2] and pred[3] in cmp_gold[3]):
                            count_correct += 1
                            break
            count_correct = min(count_correct, golds_length)
            count_wrong_from_gold = golds_length - count_correct
            count_wrong_from_pred = preds_length - count_correct
            dict_count[cur_extend_rel].append((count_correct, count_wrong_from_gold, count_wrong_from_pred))
       
    matrix = [v for _,v in dict_count.items()]

    return matrix


def get_gold_count_matrix(rel_extend_map, merged_golds, backward):
    dict = {k: [] for k in rel_extend_map.keys()}
    dict_count = {k: [] for k in rel_extend_map.keys()}

    for item in backward:
        rel = item[0][1]
        dict[rel].append(item)

    for k,v in dict.items():   
        cur_gold_rel = k
        cur_list = v
        for item in cur_list:
            count_correct = 0
            count_wrong_from_gold = 0
            count_wrong_from_pred = 0

            cur_idx = item[0][0]
            cur_merged_golds = merged_golds[cur_idx]
            if type(cur_merged_golds) == tuple:
                cur_golds = [cur_merged_golds]
            else:
                cur_golds = [gold for gold in cur_merged_golds if gold[1] == cur_gold_rel]
            cur_preds = [pred for pred in item] 

            preds_length = len(cur_preds)
            golds_length = len(cur_golds)

            if golds_length != 0:
                for pred in cur_preds:
                    for cmp_gold in cur_golds:
                        if pred[2] == '' and pred[3] == '':
                            preds_length -= 1
                            break
                        elif pred[2] == '' or pred[3] == '':
                            continue
                        elif (pred[2] in cmp_gold[2] and pred[3] in cmp_gold[3]) \
                            or (cmp_gold[2] in pred[2] and cmp_gold[3] in pred[3]) \
                            or (pred[2] in cmp_gold[2] and cmp_gold[3] in pred[3]) \
                            or (cmp_gold[2] in pred[2] and pred[3] in cmp_gold[3]):
                            count_correct += 1
                            break
            count_correct = min(count_correct, golds_length)
            count_wrong_from_gold = golds_length - count_correct
            count_wrong_from_pred = preds_length - count_correct
            dict_count[cur_gold_rel].append((count_correct, count_wrong_from_gold, count_wrong_from_pred))
       
    matrix = [v for _,v in dict_count.items()]

    return matrix

In [7]:
def word_level(rel_extend_map, matrix):
    value = [c for v in rel_extend_map.values() for c in v]
    word_count_dict = {k:[] for k in value}

    tmp_list = []

    for idx, row in enumerate(matrix):
        correct_sum = 0
        wrong_from_gold_sum = 0
        wrong_from_pred_sum = 0
        for tuple in row:
            correct_sum += tuple[0]
            wrong_from_gold_sum += tuple[1]
            wrong_from_pred_sum += tuple[2]
        if correct_sum + wrong_from_pred_sum == 0:
            P = 0.0
        else:
            P = correct_sum / (correct_sum + wrong_from_pred_sum)
        if correct_sum + wrong_from_gold_sum == 0:
            R = 0.0
        else:
            R = correct_sum / (correct_sum + wrong_from_gold_sum)
        
        if P + R == 0:
            F1 = 0.0
        else:
            F1 = 2 * P * R / (P + R)
        P_1 = 1 - P
        R_1 = 1 - R
        tmp_list.append((P, R, P_1, R_1, F1, correct_sum, wrong_from_gold_sum, wrong_from_pred_sum))


    for idx,k in enumerate(word_count_dict.keys()):
        word_count_dict[k] = tmp_list[idx]

    return word_count_dict


def word_level_gold(rel_extend_map, matrix, mode):
    word_count_dict = {k:[] for k in rel_extend_map.keys()}

    tmp_list = []

    for idx, row in enumerate(matrix):
        correct_sum = 0
        wrong_from_gold_sum = 0
        wrong_from_pred_sum = 0
        for tuple in row:
            correct_sum += tuple[0]
            wrong_from_gold_sum += tuple[1]
            wrong_from_pred_sum += tuple[2]
        if correct_sum + wrong_from_pred_sum == 0:
            P = 0.0
        else:
            P = correct_sum / (correct_sum + wrong_from_pred_sum)
        if correct_sum + wrong_from_gold_sum == 0:
            R = 0.0
        else:
            R = correct_sum / (correct_sum + wrong_from_gold_sum)
        
        if P + R == 0:
            F1 = 0.0
        else:
            F1 = 2 * P * R / (P + R)
        P_1 = 1 - P
        R_1 = 1 - R
        tmp_list.append((P, R, P_1, R_1, F1, correct_sum, wrong_from_gold_sum, wrong_from_pred_sum))


    for idx,k in enumerate(word_count_dict.keys()):
        word_count_dict[k] = tmp_list[idx]

    df = pd.DataFrame.from_dict(word_count_dict, orient='index').reset_index()
    df.columns = ['关系类型','P','R','1-P','1-R','F1','正确个数','golds中错误个数','preds中错误个数']
    df.to_excel(f'./results/word_count_dict_gold_{mode}.xlsx',index=False)

    return df


def word_dict2execl(word_count_dict, rel_extend_map, sim, mode):
    df = pd.DataFrame.from_dict(word_count_dict, orient='index').reset_index()
    df.columns = ['扩展关系词','P','R','1-P','1-R','F1','正确个数','golds中错误个数','preds中错误个数']
    extend_rel_list = df['扩展关系词'].tolist()
    gold_rel_list = [get_type(rel_extend_map, rel) for rel in extend_rel_list]
    df['关系类型'] = gold_rel_list

    cols = list(df.columns)
    cols.insert(0, cols.pop(cols.index('关系类型')))
    df = df.loc[:, cols]

    if sim != None:
        df.insert(loc=2, column='是否语义相关', value=sim)

  
    df.to_excel(f'./results/word_count_dict_{mode}.xlsx',index=False)

    return df



In [8]:
def sentence_level(rel_extend_map, sim, matrix, input_length):
    length = [len(value) for value in rel_extend_map.values()]

    sentences_dict = {k:[] for k in range(input_length)}

    l = length[0]
    c = 0
    t = -1

    for idx,key in enumerate(sentences_dict.keys()):
        if idx % 10 == 0:
            t += 1
            if idx == 0:
                last_l = 0
            else:
                last_l += l
            l = length[t]
        sim_index = last_l
        for row in matrix[last_l:last_l + l]:
            tuple = row[idx % 10]
            sentences_dict[key].append((tuple[0], tuple[1], tuple[2], sim[sim_index]))
            c += 1
            sim_index += 1
            if c == l:
                c = 0
                break
            
    return sentences_dict


def process_sim(sentences_dict, input_list2id, sim_or_not, input_length):
    sentences_count_dict = {k:[] for k in range(input_length)}

    sentence_matrix = []
    for k,v in sentences_dict.items():
        sentence_matrix.append(v)

    tmp = []

    for idx, row in enumerate(sentence_matrix):
        correct_sum = 0
        wrong_from_gold_sum = 0
        wrong_from_pred_sum = 0
        for tuple in row:
            if tuple[3] == sim_or_not:
                correct_sum += tuple[0]
                wrong_from_gold_sum += tuple[1]
                wrong_from_pred_sum += tuple[2]
        if correct_sum + wrong_from_pred_sum == 0:
            P = 0.0
        else:
            P = correct_sum / (correct_sum + wrong_from_pred_sum)
        if correct_sum + wrong_from_gold_sum == 0:
            R = 0.0
        else:
            R = correct_sum / (correct_sum + wrong_from_gold_sum)
        P_1 = 1 - P
        R_1 = 1 - R
        tmp.append((sim_or_not, P, R, P_1, R_1, correct_sum, wrong_from_gold_sum, wrong_from_pred_sum))

    for idx,k in enumerate(sentences_count_dict.keys()):
        sentences_count_dict[k] = tmp[idx]
    
    sentence = input_list2id.copy()
    for key in sentence.keys():
        sentence[key] = sentences_count_dict[input_list2id[key]]

    df = pd.DataFrame.from_dict(sentence, orient='index').reset_index()
    df.columns = ['句子','是否语义相似','P','R','1-P','1-R','正确个数','golds中错误个数','preds中错误个数']
    
    return df


def sentence_dict2execl(input_list, sentences_dict, input_length, mode):
    input_list2id = {}
    for idx, item in enumerate(input_list):
        input_list2id[item] = idx
    
    df_no = process_sim(sentences_dict, input_list2id, 0, input_length)
    df_yes = process_sim(sentences_dict, input_list2id, 1, input_length)  
    concat_df = pd.concat([df_no, df_yes], axis=0)
    concat_df_sorted = concat_df.sort_index()

    group = concat_df_sorted.groupby('句子', sort=False)
    merged_df = pd.DataFrame()
    for _, group_df in group:
        group_df = group_df.sort_values(by=['是否语义相似'], ascending=True)
        merged_df = pd.concat([merged_df, group_df], axis=0)
        
    merged_df.to_excel(f'./results/sentence_count_dict_{mode}.xlsx',index=False)
    
    return merged_df 


In [9]:
def all_level(word_count_dict):
    grouped = word_count_dict.groupby('关系类型', sort=False)
    merged_df = pd.DataFrame(columns=['关系类型', '是否语义相关', '正确个数', 'golds中错误个数', 'preds中错误个数'])

    for ent_type, group_df in grouped:
        zero_semantic_df = group_df[group_df['是否语义相关'] == 0]
        one_semantic_df = group_df[group_df['是否语义相关'] == 1]

        merged_row_0 = {
            '关系类型': ent_type,
            '是否语义相关': 0,
            '正确个数': zero_semantic_df['正确个数'].sum(),
            'golds中错误个数': zero_semantic_df['golds中错误个数'].sum(),
            'preds中错误个数': zero_semantic_df['preds中错误个数'].sum()
        }
        merged_row_1 = {
            '关系类型': ent_type,
            '是否语义相关': 1,
            '正确个数': one_semantic_df['正确个数'].sum(),
            'golds中错误个数': one_semantic_df['golds中错误个数'].sum(),
            'preds中错误个数': one_semantic_df['preds中错误个数'].sum()
        }
        
        merged_df = merged_df._append(merged_row_0, ignore_index=True)
        merged_df = merged_df._append(merged_row_1, ignore_index=True)

    return merged_df


def calculate_scores(row):
    correct = row['正确个数']
    gold_errors = row['golds中错误个数']
    preds_errors = row['preds中错误个数']
    if correct + preds_errors == 0:
        P = 0.0
    else:
        P = correct / (correct + preds_errors)
    if correct + gold_errors == 0:
        R = 0.0
    else:
        R = correct / (correct + gold_errors)
    
    P_1 = 1 - P
    R_1 = 1 - R

    return pd.Series([P, R, P_1, R_1])


def count_dict2execl(merged_df, mode):
    merged_df[['P','R','P_1','R_1']] = merged_df.apply(calculate_scores, axis=1)
    merged_df.to_excel(f'./results/all_count_dict_{mode}.xlsx',index=False)

CMeIE

In [8]:
with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/cmeie/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/cmeie/backward_zh.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/cmeie/backward_gold_zh.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]    

with open('./data/cmeie/rel_sim_human_zh.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='zh')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='zh')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='zh')

matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='zh')


SCIERC

In [9]:
with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/scierc/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/scierc/backward_en.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/scierc/backward_gold_en.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]

with open('./data/scierc/rel_sim_human_en.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='en')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='en')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='en')

matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='chatgpt_en')

Alpaca-CMeIE

In [6]:
with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/cmeie/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/cmeie/backward_alpaca_33B_zh.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/cmeie/backward_gold_alpaca_33B_zh.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]

with open('./data/cmeie/rel_sim_human_alpaca_33B_zh.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='alpaca_33B_zh')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='alpaca_33B_zh')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='alpaca_33B_zh')

matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='alpaca_33B_zh')

Alpaca-SCIERC

In [7]:
with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/scierc/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/scierc/backward_alpaca_33B_en.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/scierc/backward_gold_alpaca_33B_en.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]

with open('./data/scierc/rel_sim_human_alpaca_33B_en.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='alpaca_33B_en')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='alpaca_33B_en')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='alpaca_33B_en')

matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='alpaca_33B_en')


Llama2-SCIERC

In [23]:
with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_llama2_70B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/scierc/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/scierc/backward_llama2_70B_en.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]
 
with open('./data/scierc/backward_gold_llama2_70B_en.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]

with open('./data/scierc/rel_sim_human_llama2_70B_en.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='llama2_70B_en')
# # df_word = word_dict2execl(word_count_dict, rel_extend_map, None, mode='llama2_70B_en')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='llama2_70B_en')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='llama2_70B_en')

matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='llama2_70B_en')


ChatGLM-CMeIE

In [25]:
with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_chatglm_6B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/cmeie/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/cmeie/backward_chatglm_6B_zh.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/cmeie/backward_gold_chatglm_6B_zh.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]

with open('./data/cmeie/rel_sim_human_chatglm_6B_zh.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='chatglm_6B_zh')
# # df_word = word_dict2execl(word_count_dict, rel_extend_map, None, mode='chatglm_6B_zh')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='chatglm_6B_zh')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='chatglm_6B_zh')
matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='chatglm_6B_zh')


ChatGLM-SCIERC

In [24]:
with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_chatglm_6B_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/scierc/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/scierc/backward_chatglm_6B_en.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/scierc/backward_gold_chatglm_6B_en.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]
    
with open('./data/scierc/rel_sim_human_chatglm_6B_en.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


# matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
# word_count_dict = word_level(rel_extend_map, matrix)
# df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='chatglm_6B_en')
# # df_word = word_dict2execl(word_count_dict, rel_extend_map, None, mode='llama2_70B_en')

# sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
# df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='chatglm_6B_en')

# all_count_dict = all_level(df_word)
# count_dict2execl(all_count_dict, mode='chatglm_6B_en')
matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
df_word_gold= word_level_gold(rel_extend_map, matrix, mode='chatglm_6B_en')


GPT4-CMeIE

In [14]:
with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]
input_list = input_list[40:50]

with open('./data/cmeie/final_rel_extend_map_gpt4_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/cmeie/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]
merged_golds = merged_golds[40:50]

with open('./data/cmeie/backward_gpt4_zh.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]
    
with open('./data/cmeie/rel_sim_human_gpt4_zh.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
word_count_dict = word_level(rel_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='gpt4_zh')
# df_word = word_dict2execl(word_count_dict, rel_extend_map, None, mode='chatglm_6B_zh')

sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='gpt4_zh')

all_count_dict = all_level(df_word)
count_dict2execl(all_count_dict, mode='gpt4_zh')


GPT4-SCIERC

In [18]:
with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]
input_list = input_list[10:20]

with open('./data/scierc/final_rel_extend_map_gpt4_en.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/scierc/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]
merged_golds = merged_golds[10:20]

with open('./data/scierc/backward_gpt4_en.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]
    
with open('./data/scierc/rel_sim_human_gpt4_en.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
word_count_dict = word_level(rel_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='gpt4_en')
# df_word = word_dict2execl(word_count_dict, rel_extend_map, None, mode='llama2_70B_en')

sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='gpt4_en')

all_count_dict = all_level(df_word)
count_dict2execl(all_count_dict, mode='gpt4_en')


Baichuan2-CMeIE

In [9]:
with open('./data/cmeie/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/cmeie/final_rel_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/cmeie/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/cmeie/backward_baichuan2_13B_zh.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/cmeie/backward_gold_baichuan2_13B_zh.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]

with open('./data/cmeie/rel_sim_human_baichuan2_13B_zh.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
word_count_dict = word_level(rel_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='baichuan2_13B_zh')

sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='baichuan2_13B_zh')

all_count_dict = all_level(df_word)
count_dict2execl(all_count_dict, mode='baichuan2_13B_zh')
# matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
# df_word_gold= word_level_gold(rel_extend_map, matrix, mode='baichuan2_13B_zh')


Baichuan2-SCIERC

In [10]:
with open('./data/scierc/input_list.txt','r',encoding='utf-8') as f:
    input_list = [item.strip() for item in f.readlines()]

with open('./data/scierc/final_rel_extend_map_baichuan2_13B_en copy.json','r',encoding='utf-8') as f:
    rel_extend_map = json.load(f)

with open('./data/scierc/merged_golds.txt', 'r', encoding='utf-8') as f:
    merged_golds = [eval(line) for line in f]

with open('./data/scierc/backward_baichuan2_13B_en.txt', 'r', encoding='utf-8') as f:
    backward = [eval(line) for line in f]

with open('./data/scierc/backward_gold_baichuan2_13B_en.txt', 'r', encoding='utf-8') as f:
    backward_gold = [eval(line) for line in f]
    
with open('./data/scierc/rel_sim_human_baichuan2_13B_en copy.json','r', encoding='utf-8') as f:
    rel_sim_human = json.load(f)

input_length = len(input_list)

sim = []
for key in rel_sim_human.keys():
    for k,v in rel_sim_human[key].items():
        sim.append(v)


matrix = get_count_matrix(rel_extend_map, merged_golds, backward)
word_count_dict = word_level(rel_extend_map, matrix)
df_word = word_dict2execl(word_count_dict, rel_extend_map, sim, mode='baichuan2_13B_en')

sentences_dict = sentence_level(rel_extend_map, sim, matrix,  input_length)
df_sent = sentence_dict2execl(input_list, sentences_dict, input_length, mode='baichuan2_13B_en')

all_count_dict = all_level(df_word)
count_dict2execl(all_count_dict, mode='baichuan2_13B_en')
# matrix = get_gold_count_matrix(rel_extend_map, merged_golds, backward_gold)
# df_word_gold= word_level_gold(rel_extend_map, matrix, mode='baichuan2_13B_en')
